In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

In [2]:
df = pd.read_pickle('koh_movie_data_abridged.pkl')
df = pd.read_csv('movie_data.csv', lineterminator='\n')
# remove red dead redemption
df = df[df.id != 709579]

In [3]:
df.head()

,Unnamed: 0,popularity,id,video,vote_count,vote_average,title,release_date,original_language,original_title,genre_ids,backdrop_path,adult,overview,poster_path,genre_mapped_ids,revenue,runtime,production_companies
0,0,39.589,299534,False,13330.0,8.3,Avengers: Endgame,2019-04-24,en,Avengers: Endgame,"[12, 878, 28]",/orjiB3oUIsyz60hoEqkiGpy5CeO.jpg,False,After the devastating events of Avengers: Infi...,/or06FN3Dka5tukK1e9sl16pB3iy.jpg,"['Adventure', 'Science Fiction', 'Action']",2.797801e+09,181.0,"{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZx..."
1,1,77.767,420818,False,5880.0,7.2,The Lion King,2019-07-12,en,The Lion King,"[12, 10751]",/nRXO2SnOA75OsWhNhXstHB8ZmI3.jpg,False,"Simba idolizes his father, King Mufasa, and ta...",/2bXbqYdUdNVa8VIWXVfclP2ICtT.jpg,"['Adventure', 'Family']",1.656943e+09,118.0,"[{'id': 2, 'logo_path': '/wdrCwmRnLFJhEoH8GSfy..."
2,2,74.587,330457,False,4648.0,7.2,Frozen II,2019-11-20,en,Frozen II,"[12, 16, 10751]",/xJWPZIYOEFIjZpBL7SVBGnzRYXp.jpg,False,"Elsa, Anna, Kristoff and Olaf head far into th...",/pjeMs3yqRmFL3giJy4PMXWZTTPa.jpg,"['Adventure', 'Animation', 'Family']",1.330765e+09,104.0,"[{'id': 6125, 'logo_path': '/tVPmo07IHhBs4Huil..."
3,3,57.543,429617,False,7402.0,7.6,Spider-Man: Far from Home,2019-06-28,en,Spider-Man: Far from Home,"[28, 12, 878]",/5myQbDzw3l8K9yofUXRJ4UTVgam.jpg,False,Peter Parker and his friends go on a summer tr...,/4q2NNj4S5dG2RLF9CpXsej7yXl.jpg,"['Action', 'Adventure', 'Science Fiction']",1.131928e+09,129.0,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ..."
4,4,42.603,299537,False,9342.0,7.0,Captain Marvel,2019-03-06,en,Captain Marvel,"[28, 12, 878]",/w2PMyoyLU22YvrGK3smVM9fW1jj.jpg,False,The story follows Carol Danvers as she becomes...,/AtsgWhDnHTq68L0lLsUrCnM7TjG.jpg,"['Action', 'Adventure', 'Science Fiction']",1.128275e+09,124.0,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ..."


In [13]:
# create new dataframe with unique row for each production company for each movie
df3 = pd.DataFrame(columns=df.columns)

for i in df.index:
    row = df.loc[i]
    production_companies = row['production_companies']
    
    if type(production_companies) is dict:
        row['production'] = production_companies['name']
        df3 = df3.append(row)
    elif type(production_companies) is list:
        for prod in row['production_companies']:
            row['production'] = prod['name']
            df3 = df3.append(row)

In [14]:
df3.head()

,Unnamed: 0,popularity,id,video,vote_count,vote_average,title,release_date,original_language,original_title,genre_ids,backdrop_path,adult,overview,poster_path,genre_mapped_ids,revenue,runtime,production_companies


In [6]:
# get top 15 production companies by average revenue
df3_mean = df3.groupby('production').mean().sort_values(by='revenue', ascending=False)[:15]
df3_mean


KeyError: 'production'

In [ ]:
f, ax = plt.subplots(figsize=(20,10))
f.suptitle('Average Revenue by Production Company', fontsize=20)
plt.bar(df3_mean.index, df3_mean.revenue)
plt.xticks(rotation=90)
ax.set_ylabel('revenue')
ax.set_xlabel('production company');

In [ ]:
# get all movies belonging to top 15 production companies
time_series = df3.loc[df3.production.isin(df3_mean.index)]

In [ ]:
fig_dims = (20, 15)
fig, ax = plt.subplots(figsize=fig_dims)
line_plot = sns.lineplot(x='release_date', y='revenue', hue='production', ax=ax, data=time_series)